In [1]:
%load_ext autoreload
%autoreload 2
import os 
import pandas as pd
import numpy as np
os.chdir('..')
from src.data.make_dataset import get_preprocessed_datasets
from src.features.build_features import FeatureEngineering

In [2]:
df_bitcoin, df, df_test = get_preprocessed_datasets()

2019-11-15 16:45:55,738 - src.data.make_dataset - INFO - making final data sets from raw data
2019-11-15 16:45:56,015 - src.data.make_dataset - INFO - Shape of df_bitcoin: (2355, 2)
2019-11-15 16:45:56,015 - src.data.make_dataset - INFO - Shape of df: (4757, 91)
2019-11-15 16:45:56,077 - src.data.make_dataset - INFO - Start preprocessing dataframe
2019-11-15 16:45:56,101 - src.data.make_dataset - INFO - Replacing market_data_current_price_usd 
2019-11-15 16:45:56,122 - src.data.make_dataset - INFO - Replacing market_data_ath_usd 
2019-11-15 16:45:56,141 - src.data.make_dataset - INFO - Replacing market_data_circulating_supply 
2019-11-15 16:45:56,161 - src.data.make_dataset - INFO - Preprocessing done!


In [3]:
df.shape

(4757, 91)

In [4]:
fe = FeatureEngineering(df, df_bitcoin, df_test)

/c/Users/Sandro/repo/data_science_for_business_lmu/src/features/feature_class.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.df = pd.concat([df, df_test])


In [5]:
fe.construct_features()

2019-11-15 16:46:00,796 - src.features.feature_class - INFO - Transform numerical variable for column transaction_count
2019-11-15 16:46:00,799 - src.features.feature_class - INFO - Start filling NA values in transaction_count
2019-11-15 16:46:00,810 - src.features.feature_class - INFO - Found 2695 NA values in column transaction_count
2019-11-15 16:46:00,812 - src.features.feature_class - INFO - Filled NA values
2019-11-15 16:46:00,836 - src.features.feature_class - INFO - Transform numerical variable for column holder_count
2019-11-15 16:46:00,838 - src.features.feature_class - INFO - Start filling NA values in holder_count
2019-11-15 16:46:00,849 - src.features.feature_class - INFO - Found 2699 NA values in column holder_count
2019-11-15 16:46:00,853 - src.features.feature_class - INFO - Filled NA values
2019-11-15 16:46:00,875 - src.features.feature_class - INFO - Transform numerical variable for column market_data_current_price_usd
2019-11-15 16:46:00,877 - src.features.feature_cl

In [6]:
fe.df_features.dtypes

OBS_ID                                     int64
success                                   object
transaction_count                        float64
holder_count                             float64
market_data_current_price_usd            float64
market_data_ath_usd                      float64
market_data_total_supply                 float64
market_data_circulating_supply           float64
public_interest_stats_bing_matches       float64
labels_categories_0                        int64
labels_country_origin                      int64
labels_ico_data_country_origin             int64
labels_ico_data_hardcap_currency           int64
labels_ico_data_softcap_currency           int64
labels_ico_data_total_raised_currency      int64
dtype: object

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [17]:
X_train, y_train, X_test = fe.get_X_y()

In [9]:
X_train_, X_val, y_train_, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42)


In [10]:
X_train

array([[5.9800000e+02, 1.9300000e+02, 9.9200000e-02, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [1.5470000e+03, 9.0000000e+02, 3.3433930e-02, ..., 3.5000000e+01,
        1.5000000e+01, 1.0000000e+01],
       [3.3883000e+04, 3.8570000e+03, 3.6000000e-07, ..., 1.4000000e+01,
        8.0000000e+00, 7.0000000e+00],
       ...,
       [6.9600000e+02, 5.4400000e+02, 5.2300000e-05, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [4.8262141e+04, 1.3576306e+04, 1.5900000e-02, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [5.9498000e+04, 3.7080000e+03, 3.4701120e-02, ..., 3.5000000e+01,
        1.5000000e+01, 1.0000000e+01]])

In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(3805, 13)
(3805,)
(952, 13)
(952,)


In [12]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
y_preds = model.predict(X_val)

In [14]:
from sklearn.metrics import matthews_corrcoef


In [15]:
y_val = y_val.astype(int)

In [16]:
matthews_corrcoef(y_val, y_preds)

0.10277449599824294

In [18]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Test Set

In [31]:
X_train, y_train, X_test = fe.get_X_y()
X_train.head()

,transaction_count,holder_count,market_data_current_price_usd,market_data_ath_usd,market_data_total_supply,market_data_circulating_supply,public_interest_stats_bing_matches,labels_categories_0,labels_country_origin,labels_ico_data_country_origin,labels_ico_data_hardcap_currency,labels_ico_data_softcap_currency,labels_ico_data_total_raised_currency
0,48262.141038,13576.305982,0.001677,0.034182,6.565882e+07,8.627700e+13,23900.0,23,79,59,23,15,7
1,975773.000000,109840.000000,0.231561,43587.000000,1.000000e+09,6.008030e+15,686000.0,29,111,82,23,15,10
2,22730.000000,13914.000000,0.000717,0.043750,5.000000e+08,4.400000e+08,27500.0,29,102,76,14,8,10
3,125.000000,51.000000,0.033603,43505.000000,5.000000e+06,0.000000e+00,27500.0,26,79,59,23,15,7
4,16417.000000,6250.000000,0.000887,0.030843,1.000000e+07,0.000000e+00,703000.0,26,79,59,23,15,7


In [19]:
a = list(X_test.columns)

In [20]:
a.remove('OBS_ID')

In [21]:
a

['transaction_count',
 'holder_count',
 'market_data_current_price_usd',
 'market_data_ath_usd',
 'market_data_total_supply',
 'market_data_circulating_supply',
 'public_interest_stats_bing_matches',
 'labels_categories_0',
 'labels_country_origin',
 'labels_ico_data_country_origin',
 'labels_ico_data_hardcap_currency',
 'labels_ico_data_softcap_currency',
 'labels_ico_data_total_raised_currency']

In [22]:
X_test_values = X_test[a].values

In [23]:
X_test_values

array([[3.7600000e+02, 2.6500000e+02, 8.3971000e-04, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [7.0408000e+04, 2.3115000e+04, 5.0284000e-02, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [4.8262141e+04, 1.3576306e+04, 1.3500000e-02, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       ...,
       [4.8262141e+04, 1.3576306e+04, 3.0392000e-01, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [8.7780000e+04, 7.0199000e+04, 7.6088000e-04, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [4.8262141e+04, 1.3576306e+04, 2.8610300e-03, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00]])

In [24]:
preds_test = model.predict(X_test_values)

In [26]:
pd.DataFrame({"OBS_ID": X_test['OBS_ID'],
             "success": preds_test}).to_csv('data/processed/submission05', index=None)

In [27]:
pd.read_csv('data/processed/submission05')

,OBS_ID,success
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
996,997,0
997,998,0
998,999,0
999,1000,0


In [60]:
b = pd.read_csv('data/processed/submission03')

In [61]:
a.success.value_caounts()

AttributeError: 'Series' object has no attribute 'value_caounts'

In [62]:
b.success.value_counts()

0    982
1     19
Name: success, dtype: int64